In [1]:
import time
from collections import defaultdict
from prettytable import PrettyTable

In [2]:
class FPTreeNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

    def display(self, ind=1):
        print('  ' * ind, f'{self.item}: {self.count}')
        for child in self.children.values():
            child.display(ind + 1)

In [3]:
def build_fp_tree(transactions, min_support):
    header_table = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            header_table[item] += 1
    header_table = {k: v for k, v in header_table.items() if v >= min_support}
    if not header_table:
        return None, None
    for key in header_table:
        header_table[key] = [header_table[key], None]
    root = FPTreeNode(None, 1, None)
    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in header_table]
        filtered_transaction.sort(key=lambda x: header_table[x][0], reverse=True)
        insert_tree(filtered_transaction, root, header_table)
    return root, header_table

def insert_tree(items, node, header_table):
    if items:
        first_item = items[0]
        if first_item in node.children:
            node.children[first_item].increment(1)
        else:
            new_node = FPTreeNode(first_item, 1, node)
            node.children[first_item] = new_node
            if header_table[first_item][1] is None:
                header_table[first_item][1] = new_node
            else:
                current = header_table[first_item][1]
                while current.link is not None:
                    current = current.link
                current.link = new_node
        insert_tree(items[1:], node.children[first_item], header_table)

In [4]:
def mine_fp_tree(header_table, min_support, prefix, frequent_itemsets):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])
    for base_item, (count, node) in sorted_items:
        new_prefix = prefix.copy()
        new_prefix.add(base_item)
        frequent_itemsets.append((new_prefix, count))
        conditional_pattern_base = []
        while node is not None:
            path = []
            parent = node.parent
            while parent is not None and parent.item is not None:
                path.append(parent.item)
                parent = parent.parent
            path.reverse()
            for _ in range(node.count):
                conditional_pattern_base.append(path)
            node = node.link
        conditional_tree, conditional_header = build_fp_tree(conditional_pattern_base, min_support)
        if conditional_header is not None:
            print(f"\nConditional FP-tree for prefix {new_prefix}:")
            conditional_tree.display()
            mine_fp_tree(conditional_header, min_support, new_prefix, frequent_itemsets)

In [5]:
transactions = []
with open('b.txt', 'r') as file:
    for line in file:
        transactions.append(line.strip().split())

In [6]:
print(f"Total number of transactions: {len(transactions)}")

Total number of transactions: 6


In [7]:
min_support = 3
print(f"Minimum support count: {min_support}")


Minimum support count: 3


In [8]:
start_time = time.time()
root, header_table = build_fp_tree(transactions, min_support)
if root is not None:
    print("\nInitial FP-tree:")
    root.display()
    frequent_itemsets = []
    mine_fp_tree(header_table, min_support, set(), frequent_itemsets)
    end_time = time.time()
    print("\nFrequent Itemsets:")
    table = PrettyTable(["Itemset", "Support"])
    for itemset, support in frequent_itemsets:
        table.add_row([list(itemset), support])
    print(table)
    print(f"\nTotal frequent itemsets: {len(frequent_itemsets)}")
    print(f"Computation time: {end_time - start_time:.2f} seconds")
else:
    print("No frequent itemsets found.")


Initial FP-tree:
   None: 1
     Bread: 5
       Milk: 1
       Diaper: 4
         Beer: 1
         Milk: 3
           Beer: 1
     Diaper: 1
       Milk: 1
         Beer: 1

Conditional FP-tree for prefix {'Beer'}:
   None: 1
     Diaper: 3

Conditional FP-tree for prefix {'Milk'}:
   None: 1
     Bread: 4
       Diaper: 3
     Diaper: 1

Conditional FP-tree for prefix {'Milk', 'Diaper'}:
   None: 1
     Bread: 3

Conditional FP-tree for prefix {'Diaper'}:
   None: 1
     Bread: 4

Frequent Itemsets:
+-----------------------------+---------+
|           Itemset           | Support |
+-----------------------------+---------+
|           ['Beer']          |    3    |
|      ['Beer', 'Diaper']     |    3    |
|          ['Bread']          |    5    |
|           ['Milk']          |    5    |
|      ['Milk', 'Bread']      |    4    |
|      ['Milk', 'Diaper']     |    4    |
| ['Milk', 'Bread', 'Diaper'] |    3    |
|          ['Diaper']         |    5    |
|     ['Bread', 'Diaper']     